# Karger algorithm

## Support objects

In [401]:
class Node:
    def __init__(self, tag: int):
        self.tag = tag
        self.adjacencyList = []

class Graph:
    def __init__(self):
        self.nodes = defaultdict(Node)

    def createNodes(self, nums: int):
        for i in range(1, nums+1):
            self.nodes[i] = Node(i)

    def addNode(self, tag:int, adjTag:int):
        self.nodes[tag].adjacencyList.append(self.nodes[adjTag])
        self.nodes[adjTag].adjacencyList.append(self.nodes[tag]) # Graph is undirected

    def buildGraph(self):
        self.createNodes(len(dataset))
        for v in dataset:
            for i in range (1, len(v)):
                self.addNode(v[0], v[i])

## Reading the input

In [402]:
from collections import defaultdict
def readInput(path: str):
    lines = open(path, "r").read().split("\n")
    input = []
    for line in range(len(lines)-1):
        input.append(list(map(int, lines[line].split()))) # Transform list of strings to list of integers

    return input

## Karger implementation

In [403]:
import random
def karger(graph):
    # Select two linked nodes (arch)
    n1 = graph.nodes[random.choice(range(1,len(graph.nodes)+1))]
    n2 = n1.adjacencyList[random.choice(range(len(n1.adjacencyList)))]
    print(n1.tag,n2.tag)
    # Contraction

In [404]:
dataset = readInput("mincut_dataset/input_random_1_6.txt")
graph = Graph()
graph.buildGraph()
result = karger(graph)

6 5
